# **부산 지역 시간별 데이터 정제**
- ~~시간별 데이터 크롤링~~
- 5시간 측정치 평균값 : 엑셀에서
-

In [ ]:
#for문으로 여러 csv 파일 불러오기
import os

path = 'data/fin/'
file_list = os.listdir(path)
files = [file for file in file_list if file.endswith('.csv')]
files

In [ ]:
#for문으로 불러온 csv 파일 -> DataFrame -> concat
import pandas as pd
df = pd.DataFrame()
for i in files:
    data = pd.read_csv(path+i)
    df = pd.concat([df, data])

df = df.reset_index(drop=True)
df

In [ ]:
time = pd.DataFrame(df['일시'])
time

In [ ]:
df = df.transpose()
df

In [ ]:
df1 = df.iloc[1:, :]
df1.head()

In [ ]:
df1 = df1.transpose()
df1

In [ ]:
df1['주소'] = df1.index
df1

In [ ]:
df1= df1.reset_index(drop=True)
df1

In [ ]:
re = pd.DataFrame(df1['주소'].apply(lambda v: v.split()[:2]).tolist(),columns=('시도', '군구'))
re

In [ ]:
re['시도'] = '부산광역시'
re

In [ ]:
re['주소'] = re.apply(lambda r: r['시도']+' '+r['군구'], axis=1)
re

In [ ]:
result = pd.merge(re, df1, how='inner', left_index=True, right_index=True)
result

In [ ]:
result = result.drop(["주소_y"], axis=1)
result

In [ ]:
result.rename(columns={"주소_x":"주소"})

In [ ]:
num_df = result.iloc[:, 3:]
num_df

In [ ]:
cols = num_df.columns
cols

In [ ]:
for i in range(len(cols)):
    for j in range(len(num_df)):
        if num_df[i][j] == '-':
            num_df[i][j] = 0

In [ ]:
num_df = num_df.astype('float')
num_df

In [ ]:
num_df.dtypes

In [ ]:
re = pd.merge(re, num_df, how='inner', left_index=True, right_index=True)
re

In [ ]:
result = re.drop(["시도", "군구"], axis=1)
result

In [ ]:
#실수 소수점 2째자리까지
pd.options.display.float_format = '{:.2f}'.format
f = result.groupby(['주소']).mean()
f

In [ ]:
ff = f.transpose()
ff

In [ ]:
result = pd.merge(ff, time, how='inner', left_index=True, right_index=True)
result

In [ ]:
result.dtypes

In [ ]:
result.to_csv('data/fin/_busan_6m.csv', index=False)

### **TO DO**
- csv 파일 열어서 부산 지역구별 5시간 평균값 구해서 저장(_busan_수치정규화.csv) 파일